# STEM Algorithm

## Step 1

In [1]:
import numpy as np
import pyomo.environ as pyo

#### Input

In [2]:
I, J, K = 3, 4, 3

Ckj = [
    [10, 30, 50, 100],
    [1, 1, 0, 0],
    [1, 4, 6, 2]
]
Aij = [
    [5, 3, 2, 0],
    [0, 0, 3, 8],
    [2, 3, 4, 6]
]
Bi = [240, 320, 180]

In [3]:
f = lambda k, *x: np.dot(x, np.array(Ckj)[k].reshape(-1, 1))

In [4]:
model = pyo.ConcreteModel()

model.i = pyo.RangeSet(0, I-1)
model.j = pyo.RangeSet(0, J-1)
model.k = pyo.RangeSet(0, K-1)

model.a = pyo.Param(model.i, model.j, initialize=dict(np.ndenumerate(Aij)))
model.b = pyo.Param(model.i, initialize=dict(enumerate(Bi)))
model.c = pyo.Param(model.k, model.j, initialize=dict(np.ndenumerate(Ckj)))

model.x = pyo.Var(model.j, domain=pyo.NonNegativeReals)


def cons_rule(model, i):
    return sum(model.a[i, j]*model.x[j] for j in model.j) <= model.b[i]
model.cons = pyo.Constraint(model.i, rule=cons_rule)

In [5]:
income_matrix = np.empty((K, K))

In [6]:
for k in range(K):
    model.obj = pyo.Objective(
        expr=sum(model.c[k, j]*model.x[j] for j in model.j),
        sense=pyo.maximize
    )

    opt = pyo.SolverFactory('cplex')
    opt.solve(model)

    x = [pyo.value(model.x[j]) for j in model.j]
    obj = pyo.value(model.obj)
    print(obj)
    
    del model.obj

    matrix = np.array([])
    for z in range(K):
        if z == k:
            income_matrix[k, z] = obj
            continue  
        income_matrix[k, z] = f(z, *x)

3000.0
66.66666666666667
270.0


In [7]:
alpha = np.empty(K)

In [8]:
for k in range(K):
    f_star = income_matrix[k, k]
    f_min = min(income_matrix[:, k])
    c = 1/np.sqrt(sum(np.array(Ckj)[k]**2))
    if c >= 1:
        c = np.floor(c)

    if income_matrix[k, k] > 0:
        alpha[k] = (f_star-f_min)*c/(f_star)
    else:
        alpha[k] = (f_min-f_star)*c/(f_min)


In [9]:
pi = alpha/sum(alpha)

In [10]:
model.pi = pyo.Param(model.k, initialize=pi, mutable=True)
model.lambd = pyo.Var(domain=pyo.NonNegativeReals)
model.obj = pyo.Objective(expr=model.lambd)

model.cons_dm = pyo.ConstraintList()
for k in model.k:
    model.cons_dm.add(
        expr=model.lambd>=(income_matrix[k, k]-sum(model.c[k, j]*model.x[j] for j in model.j))*model.pi[k]
    )

In [11]:
opt = pyo.SolverFactory('cplex')
opt.solve(model)

x = [pyo.value(model.x[j]) for j in model.j]
res = [f(k, *x) for k in range(K)]

for func in res:
    print(func[0])

1802.6838369423806
59.86580815288097
239.59742445864293


## Step 2

#### Input

In [12]:
func_old = np.array([1802.6838369423806, 59.86580815288097, 239.59742445864293])
adjustment_data = {1: 9.8681}

In [13]:
adjustment_key = list(adjustment_data.keys())[0]
adjustment_value = adjustment_data[adjustment_key]

for k in range(K):
    model.pi[k] = 1/(I-1)
model.pi[adjustment_key] = 0

for k in range(K):
    if k == adjustment_key:
        model.cons_dm.add(
            expr=sum(model.c[k, j]*model.x[j] for j in model.j)>=(func_old[k]-adjustment_value)
        )
        continue
    model.cons_dm.add(
        expr=sum(model.c[k, j]*model.x[j] for j in model.j)>=func_old[k]
    )

In [14]:
opt = pyo.SolverFactory('cplex')
opt.solve(model)

x = [pyo.value(model.x[j]) for j in model.j]
res = [f(k, *x) for k in range(K)]

for func in res:
    print(func[0])

1894.316194085238
49.997708152880975
239.59742445864293
